In [1]:
import time
import numpy as np
from matplotlib import pyplot
from matplotlib import pyplot as plt
import pickle
import pandas as pd
import IPython.Display 

In [2]:
data_list=['Barclays-Noida-3.2-Jan', 'Barclays-Noida-3.2-Feb', 'Barclays-Noida-3.2-March','Barclays-Noida-3.2-April']

for month in range(len(data_list)):
    month_data=pickle.load(open('../../processed_data/'+data_list[month]+'.p', "rb" ))
    if month==0:
        data=month_data.copy()
    else:
        data=pd.concat([data,month_data])
data=data.drop(['Country','Department','Resource Type'],axis=1)

In [3]:
data=data.groupby([data.floor,data.seatName,pd.Grouper(key='Datetime',freq='H')])
data=data['Space'].agg([np.sum])
data.reset_index(inplace=True)
data['isOccupied']=[0 if x <3 else 1 for x in data['sum']]
data=data.drop(['sum'],axis=1)
floor_data=data.groupby([data.floor,pd.Grouper(key='Datetime',freq='H')])['isOccupied'].agg([np.sum])
floor_data.reset_index(inplace=True)
one_floor=floor_data.loc[floor_data.floor=='Level 9']
one_floor.set_index('Datetime',inplace=True)
one_floor.index.freq='H'
one_floor.head(3)

,floor,sum
Datetime,,
2018-01-29 00:00:00,Level 9,23
2018-01-29 01:00:00,Level 9,21
2018-01-29 02:00:00,Level 9,14


In [21]:
# transform to supervised learning
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in reversed(range(1, lag+1))]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.dropna(inplace=True)
    return df

In [22]:
one_floor

,floor,sum
Datetime,,
2018-01-29 00:00:00,Level 9,23
2018-01-29 01:00:00,Level 9,21
2018-01-29 02:00:00,Level 9,14
2018-01-29 03:00:00,Level 9,3
2018-01-29 04:00:00,Level 9,3
2018-01-29 05:00:00,Level 9,5
2018-01-29 06:00:00,Level 9,3
2018-01-29 07:00:00,Level 9,3
2018-01-29 08:00:00,Level 9,3


In [23]:
lag=3
sv_floor_data = timeseries_to_supervised(one_floor['sum'], lag)
sv_floor_data.head(3)

,sum,sum,sum,sum
Datetime,,,,
2018-01-29 03:00:00,23.0,21.0,14.0,3
2018-01-29 04:00:00,21.0,14.0,3.0,3
2018-01-29 05:00:00,14.0,3.0,3.0,5


In [25]:
columns=[]
for x in range(1,lag+1):
    columns.append('x'+str(x))
columns.append('y')
sv_floor_data.columns=columns
df=pd.merge(sv_floor_data,one_floor, right_index=True, left_index=True,  how='left')
#df.drop(['sum'],axis=1,inplace=True)
df.head(3)

,x1,x2,x3,y,floor,sum
Datetime,,,,,,
2018-01-29 03:00:00,23.0,21.0,14.0,3,Level 9,3
2018-01-29 04:00:00,21.0,14.0,3.0,3,Level 9,3
2018-01-29 05:00:00,14.0,3.0,3.0,5,Level 9,5


In [ ]:
date_split='2018-04-24'
train,test=one_floor.loc[one_floor.index<date_split],one_floor.loc[one_floor.index>=date_split]